# Recommendation System

In this example we are creating a movie recommendation system by extractung feature vectors using PaddlePaddle, importing movie vectors into Milvus, and then searching in Milvus and Redis.

## Data
In this project, we use [MovieLens 1M](https://grouplens.org/datasets/movielens/1m/). This dataset contains 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users. 

We use the following files:
- movies.dat: Contains movie information.
- movie_vectors.txt: Contains movie vectors that can be imported to Milvus easily.

File structure:

 - movMovieID::Title::Genres   

     - Titles are identical to titles provided by the IMDB (includingyear of release)
 
     - Genres are pipe-separated

     - Some MovieIDs do not correspond to a movie due to accidental duplicate entries and/or test entries
 
    - Movies are mostly entered by hand, so errors and inconsistencies may exist



## Requirements

Due to package constraints, this notebook needs to be run using Python 3.6/3.7 . It is recommended that you use a virtual enviroment like Conda, instructions for installing Conda can be found [here](https://conda.io/projects/conda/en/latest/user-guide/install/index.html). 

Currently, there is a dirty workaround that you can use for python 3.8. When installing the `requirements.txt`, pip will fail to install`sentencepiece`. If you rerun the notebook after the install fails and avoid redownloading the packages, the rest of the notebook should run without any hiccups.

|  Packages |  Servers |
| --------------- | -------------- |
| pymilvus-orm==2.0.0rc1 | milvus-2.0.0-rc1 |
| redis           | redis          |
| paddle_serving_app |
| paddlepaddle==2.1.1 |


We have included a requirements.txt file in order to easily satisfy the required packages. 

## Up and Running

### Installing Packages
Install the required python packages with `requirements.txt`. If using Python 3.8, look at workaround in the Requirements section.

In [1]:
! pip install -r requirements.txt

  Using cached pymilvus_orm-2.0.0rc1-py3-none-any.whl (35 kB)
  Using cached paddle_serving_app-0.6.2-py3-none-any.whl (52 kB)
  Using cached paddlepaddle-2.1.1-cp36-cp36m-macosx_10_6_intel.whl (72.1 MB)
  Using cached redis-3.5.3-py2.py3-none-any.whl (72 kB)
  Using cached pymilvus-2.0.0rc1-py3-none-any.whl (70 kB)
  Using cached pandas-1.1.5-cp36-cp36m-macosx_10_9_x86_64.whl (10.2 MB)
  Using cached protobuf-3.17.3-cp36-cp36m-macosx_10_9_x86_64.whl (1.0 MB)
  Using cached Pillow-8.3.1-cp36-cp36m-macosx_10_10_x86_64.whl (2.9 MB)
  Using cached gast-0.5.0-py3-none-any.whl (10 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached numpy-1.19.5-cp36-cp36m-macosx_10_9_x86_64.whl (15.6 MB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached grpcio-1.37.1-cp36-cp36m-macosx_10_10_x86_64.whl (3.8 MB)
  Using cached ujson-4.0.2-cp36-cp

### Starting Milvus Server

This demo uses Milvus 2.0 Standalone with docker-compose, please refer to [Install Milvus 2.0](https://milvus.io/docs/v2.0.0/install_standalone-docker.md) for other installation options (on Kubernetes or use Milvus Cluster).

In [2]:
!wget https://raw.githubusercontent.com/milvus-io/milvus/master/deployments/docker/standalone/docker-compose.yml -O docker-compose.yml
!docker-compose up -d

--2021-07-09 16:36:06--  https://raw.githubusercontent.com/milvus-io/milvus/master/deployments/docker/standalone/docker-compose.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1297 (1.3K) [text/plain]
Saving to: ‘docker-compose.yml’

docker-compose.yml  100%[===================>]   1.27K  --.-KB/s    in 0s      

2021-07-09 16:36:06 (32.6 MB/s) - ‘docker-compose.yml’ saved [1297/1297]

[+] Running 0/0
 ⠋ Network milvus  Creating                                                0.0s
[+] Running 0/1
 ⠙ Network milvus  Creating                                                0.1s
[+] Running 0/1
 ⠹ Network milvus  Creating                                                0.2s
[+] Running 0/1
 ⠸ Network milvus  Creating                                            

[+] Running 2/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Starting                                   0.6s
 ⠿ Container milvus-etcd        Starting                                   0.6s
 ⠿ Container milvus-standalone  Created                                    0.1s
[+] Running 2/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Starting                                   0.7s
 ⠿ Container milvus-etcd        Starting                                   0.7s
 ⠿ Container milvus-standalone  Created                                    0.1s
[+] Running 2/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Starting                                   0.8s
 ⠿ Container milvus-etcd        Starting                                   0.8s
 ⠿ Container milvus-standalone  Created                                 

[+] Running 2/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Starting                                   2.7s
 ⠿ Container milvus-etcd        Starting                                   2.7s
 ⠿ Container milvus-standalone  Created                                    0.1s
[+] Running 2/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Starting                                   2.8s
 ⠿ Container milvus-etcd        Starting                                   2.8s
 ⠿ Container milvus-standalone  Created                                    0.1s
[+] Running 2/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Starting                                   2.9s
 ⠿ Container milvus-etcd        Starting                                   2.9s
 ⠿ Container milvus-standalone  Created                                 

[+] Running 3/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Started                                    3.6s
 ⠿ Container milvus-etcd        Started                                    3.4s
 ⠿ Container milvus-standalone  Starting                                   4.6s
[+] Running 3/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Started                                    3.6s
 ⠿ Container milvus-etcd        Started                                    3.4s
 ⠿ Container milvus-standalone  Starting                                   4.7s
[+] Running 3/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Started                                    3.6s
 ⠿ Container milvus-etcd        Started                                    3.4s
 ⠿ Container milvus-standalone  Starting                                

[+] Running 3/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Started                                    3.6s
 ⠿ Container milvus-etcd        Started                                    3.4s
 ⠿ Container milvus-standalone  Starting                                   6.7s
[+] Running 3/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Started                                    3.6s
 ⠿ Container milvus-etcd        Started                                    3.4s
 ⠿ Container milvus-standalone  Starting                                   6.8s
[+] Running 3/4
 ⠿ Network milvus               Created                                    4.1s
 ⠿ Container milvus-minio       Started                                    3.6s
 ⠿ Container milvus-etcd        Started                                    3.4s
 ⠿ Container milvus-standalone  Starting                                

### Starting Redis Server
We are using Redis as a metadata storage service. Code can easily be modified to use a python dictionary, but that usually does not work in any use case outside of quick examples. We need a metadata storage service in order to be able to be able to map between embeddings and the corresponding data.

In [3]:
!docker run  --name redis -d -p 6379:6379 redis

23e57c4ac4ba221189c3b51284e76b5663ba19122062df88a213b5dbfce69406


### Confirm Running Servers

In [4]:
! docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED          STATUS                             PORTS                                           NAMES
23e57c4ac4ba   redis          "docker-entrypoint.s…"   4 seconds ago    Up 3 seconds                       0.0.0.0:6379->6379/tcp, :::6379->6379/tcp       redis
582c48fa905a   77552fa61aeb   "/tini -- milvus run…"   30 seconds ago   Up 21 seconds                      0.0.0.0:19530->19530/tcp, :::19530->19530/tcp   milvus-standalone
a4d210b6f249   61ad63875109   "etcd -listen-peer-u…"   30 seconds ago   Up 26 seconds (health: starting)   2379-2380/tcp                                   milvus-etcd
f32fac178bc3   b849e0c26c4c   "/usr/bin/docker-ent…"   30 seconds ago   Up 26 seconds (health: starting)   9000/tcp                                        milvus-minio


### Downloading Pretrained Models
This PaddlePaddle model is used to transform user information into vectors.

In [5]:
!wget https://paddlerec.bj.bcebos.com/aistudio/user_vector.tar.gz --no-check-certificate
!mkdir movie_recommender
!mkdir movie_recommender/user_vector_model
!tar xf user_vector.tar.gz -C movie_recommender/user_vector_model/
!rm user_vector.tar.gz

--2021-07-09 16:36:55--  https://paddlerec.bj.bcebos.com/aistudio/user_vector.tar.gz
Resolving paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)... 103.235.46.61
Connecting to paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33650924 (32M) [application/x-gzip]
Saving to: ‘user_vector.tar.gz’

user_vector.tar.gz  100%[===================>]  32.09M  2.62MB/s    in 39s     

2021-07-09 16:37:36 (844 KB/s) - ‘user_vector.tar.gz’ saved [33650924/33650924]



### Downloading Data

In [6]:
# Download movie information
!wget -P movie_recommender https://paddlerec.bj.bcebos.com/aistudio/movies.dat --no-check-certificate
# Download movie vecotrs
!wget -P movie_recommender https://paddlerec.bj.bcebos.com/aistudio/movie_vectors.txt --no-check-certificate

--2021-07-09 16:37:42--  https://paddlerec.bj.bcebos.com/aistudio/movies.dat
Resolving paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)... 103.235.46.61
Connecting to paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171308 (167K) [application/octet-stream]
Saving to: ‘movie_recommender/movies.dat’

movies.dat          100%[===================>] 167.29K  94.2KB/s    in 1.8s    

2021-07-09 16:37:45 (94.2 KB/s) - ‘movie_recommender/movies.dat’ saved [171308/171308]

--2021-07-09 16:37:45--  https://paddlerec.bj.bcebos.com/aistudio/movie_vectors.txt
Resolving paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)... 103.235.46.61
Connecting to paddlerec.bj.bcebos.com (paddlerec.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1095505 (1.0M) [text/plain]
Saving to: ‘movie_recommender/movie_vectors.txt’

movie_vectors.txt   100%[===================>]   1

## Code Overview


### Importing Movies into Milvus

#### 1. Connectings to Milvus and Redis
Both servers are running as Docker containers on the localhost with their corresponding default ports.

In [7]:
from pymilvus_orm import *
import redis

connections.connect()
# connections.connect("default", host="localhost", port="19530")
r = redis.StrictRedis(host="localhost", port=6379) 

#### 2. Loading Movies into Redis
We begin by loading all the movie files into redis. 

In [8]:
import json
import codecs

#1::Toy Story (1995)::Animation|Children's|Comedy
def process_movie(lines, redis_cli):
    for line in lines:
        if len(line.strip()) == 0:
            continue
        tmp = line.strip().split("::")
        movie_id = tmp[0]
        title = tmp[1]
        genre_group = tmp[2]
        tmp = genre_group.strip().split("|")
        genre = tmp
        movie_info = {"movie_id" : movie_id,
                "title" : title,
                "genre" : genre
                }
        redis_cli.set("{}##movie_info".format(movie_id), json.dumps(movie_info))
        
with codecs.open("movie_recommender/movies.dat", "r",encoding='utf-8',errors='ignore') as f:
        lines = f.readlines()
        process_movie(lines, r)

#### 3. Creating Partition and Collection in Milvus

In [9]:
COLLECTION_NAME = 'demo_films'
PARTITION_NAME = 'Movie'

pk = FieldSchema(name='pk', dtype=DataType.INT64, is_primary=True, auto_id=False)
field = FieldSchema(name='vec', dtype=DataType.FLOAT_VECTOR, dim=32)
schema = CollectionSchema(fields=[pk, field], description="movie recommendation: demo films")

if utility.get_connection().has_collection(COLLECTION_NAME): # drop the same collection created before
    collection = Collection(COLLECTION_NAME)
    collection.drop()
else:
    collection = Collection(name=COLLECTION_NAME, schema=schema)
    partition = collection.create_partition(PARTITION_NAME)
    print("Collection & partition are successfully created.")

Collection & partition are successfully created.


#### 4. Getting Embeddings and IDs
The vectors in `movie_vectors.txt` are obtained from the `user_vector_model` downloaded above. So we can directly get the vectors and the IDs by reading the file.

In [10]:
def get_vectors():
    with codecs.open("movie_recommender/movie_vectors.txt", "r", encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()
    ids = [int(line.split(":")[0]) for line in lines]
    embeddings = []
    for line in lines:
        line = line.strip().split(":")[1][1:-1]
        str_nums = line.split(",")
        emb = [float(x) for x in str_nums]
        embeddings.append(emb)
    return ids, embeddings

ids, embeddings = get_vectors()

#### 4. Importing Vectors into Milvus
Import vectors into the partition **Movie** under the collection **demo_films**.

In [11]:
if collection.num_entities != 0:
    print(COLLECTION_NAME + " is not empty!")  
else:
    mr = collection.insert(data=[ids,embeddings], partition_name=PARTITION_NAME)

print("Record count in collection: " + str(collection.num_entities))
# print(str(len(mr.primary_keys)) + " ids:\n", mr.primary_keys[:10])

Record count in collection: 3883


### Build Index

In [12]:
# Flush collection with inserted vectors to disk
# utility.get_connection().flush([COLLECTION_NAME])

index_param = {
    "metric_type": "L2",
    "index_type":"IVF_FLAT",
    "params":{"nlist":128}
}

collection.create_index(field_name="vec", index_params=index_param)

Status(code=0, message='')

### Recalling Vectors in Milvus
#### 1. Genarating User Embeddings
Pass in the gender, age and occupation of the user we want to recommend. **user_vector_model** model will generate the corresponding user vector.
Occupation is chosen from the following choices:
*  0:  "other" or not specified
*  1:  "academic/educator"
*  2:  "artist"
*  3:  "clerical/admin"
*  4:  "college/grad student"
*  5:  "customer service"
*  6:  "doctor/health care"
*  7:  "executive/managerial"
*  8:  "farmer"
*  9:  "homemaker"
*  10:  "K-12 student"
*  11:  "lawyer"
*  12:  "programmer"
*  13:  "retired"
*  14:  "sales/marketing"
*  15:  "scientist"
*  16:  "self-employed"
*  17:  "technician/engineer"
*  18:  "tradesman/craftsman"
*  19:  "unemployed"
*  20:  "writer"

In [13]:
import numpy as np
from paddle_serving_app.local_predict import LocalPredictor

class RecallServerServicer(object):
    def __init__(self):
        self.uv_client = LocalPredictor()
        self.uv_client.load_model_config("movie_recommender/user_vector_model/serving_server_dir") 
        
    def hash2(self, a):
        return hash(a) % 1000000

    def get_user_vector(self):
        dic = {"userid": [], "gender": [], "age": [], "occupation": []}
        lod = [0]
        dic["userid"].append(self.hash2('0'))
        dic["gender"].append(self.hash2('M'))
        dic["age"].append(self.hash2('23'))
        dic["occupation"].append(self.hash2('6'))
        lod.append(1)

        dic["userid.lod"] = lod
        dic["gender.lod"] = lod
        dic["age.lod"] = lod
        dic["occupation.lod"] = lod
        for key in dic:
            dic[key] = np.array(dic[key]).astype(np.int64).reshape(len(dic[key]),1)
        fetch_map = self.uv_client.predict(feed=dic, fetch=["save_infer_model/scale_0.tmp_1"], batch=True)
        return fetch_map["save_infer_model/scale_0.tmp_1"].tolist()[0]

recall = RecallServerServicer()
user_vector = recall.get_user_vector()

2021-07-09 16:39:29,170 - INFO - LocalPredictor load_model_config params: model_path:movie_recommender/user_vector_model/serving_server_dir, use_gpu:False, gpu_id:0, use_profile:False, thread_num:1, mem_optim:True, ir_optim:False, use_trt:False, use_lite:False, use_xpu:False, precision:fp32, use_calib:False, use_mkldnn:False, mkldnn_cache_capacity:0, mkldnn_op_list:None, mkldnn_bf16_op_list:None, use_feed_fetch_ops:False, 


#### 2. Searching
Pass in the user vector, and then recall vectors in the previously imported data collection and partition.

In [14]:
collection.load() # load collection memory before search

topK = 20
SEARCH_PARAM = {
    "metric_type":"L2",
    "params":{"nprobe": 20},
    }
results = collection.search([user_vector],"vec",param=SEARCH_PARAM, limit=topK, expr=None, output_fields=None)

#### 3. Returning Information by IDs

In [15]:
I = []
for x in results:
    for y in x.ids:
        I.append(y)
        
recall_results = []
for x in I:
    recall_results.append(r.get("{}##movie_info".format(x)).decode('utf-8'))
recall_results

['{"movie_id": "760", "title": "Stalingrad (1993)", "genre": ["War"]}',
 '{"movie_id": "1350", "title": "Omen, The (1976)", "genre": ["Horror"]}',
 '{"movie_id": "1258", "title": "Shining, The (1980)", "genre": ["Horror"]}',
 '{"movie_id": "632", "title": "Land and Freedom (Tierra y libertad) (1995)", "genre": ["War"]}',
 '{"movie_id": "3007", "title": "American Movie (1999)", "genre": ["Documentary"]}',
 '{"movie_id": "2086", "title": "One Magic Christmas (1985)", "genre": ["Drama", "Fantasy"]}',
 '{"movie_id": "1303", "title": "Man Who Would Be King, The (1975)", "genre": ["Adventure"]}',
 '{"movie_id": "3920", "title": "Faraway, So Close (In Weiter Ferne, So Nah!) (1993)", "genre": ["Drama", "Fantasy"]}',
 '{"movie_id": "1051", "title": "Trees Lounge (1996)", "genre": ["Drama"]}',
 '{"movie_id": "1605", "title": "Excess Baggage (1997)", "genre": ["Adventure", "Romance"]}',
 '{"movie_id": "652", "title": "301, 302 (1995)", "genre": ["Mystery"]}',
 '{"movie_id": "1275", "title": "High

## Conclusion

After completing the recall service, the results can be further sorted using the **movie_recommender** model, and then the movies with high similarity scores can be recommended to users. You can try this deployable recommendation system using this [quick start](QUICK_START.md).